# Find one top representative single-cell crop visualize per top healthy and failing coefficients

The features with the highest coefficients per channel per cell type per identified *manually*. We do not use correlation features due to having to use two features and those are harder to visualize.

We first filter the plate 4 data frame to only include isolated cells (0 cell neighbors adjacent) and then filter out any single-cell that is too close to an edge (based on crop_size).

### **Top healthy features per channel**

| Feature | Coefficient |
|-----------------|-----------------|
| Nuclei_Intensity_MeanIntensityEdge_**Hoechst** | 1.6491632293619 |
| Cytoplasm_Texture_AngularSecondMoment_**ER**_3_01_256 | 0.251023308427374 |
| Nuclei_Intensity_IntegratedIntensity_**PM** | 0.647266129401344 |
| Nuclei_RadialDistribution_MeanFrac_**Mitochondria**_3of4 | 0.338620308838192 |
| Nuclei_Intensity_MinIntensity_**Actin** | 0.34104254666359 |

### **Top failing features per channel**

| Feature | Coefficient |
|-----------------|-----------------|
| Nuclei_Intensity_StdIntensityEdge_**Hoechst** | 0.731721019099269 |
| Cells_Intensity_IntegratedIntensityEdge_**ER** | 0.260370773712069 |
| Cells_Texture_InfoMeas1_**PM**_3_00_256 | 0.277759767032262 |
| Nuclei_Intensity_LowerQuartileIntensity_**Mitochondria** | 0.280095597808861 |
| Cells_Intensity_IntegratedIntensityEdge_**Actin** | 1.1894472062255 |


In [1]:
import pathlib
from pprint import pprint

import cv2
import pandas as pd
from typing import List

In [2]:
# Function for formatting min/max row data frames into dictionaries
def create_sc_dict(dfs: List[pd.DataFrame], names: List[str]) -> dict:
    """Format lists of data frames and names into a dictionary with all relevant metadata to find single-cell images.

    Args:
        dfs (List[pd.DataFrame]): List of data frames each containing a single cell and relevant metadata.
        names (List[str]): List of names corresponding to the data frames.

    Returns:
        dict: Dictionary containing info relevant for finding single-cell crops.
    """
    sc_dict = {}
    for df, name in zip(dfs, names):
        for i, (_, row) in enumerate(df.iterrows()):
            key = f"{name}"
            sc_dict[key] = {
                "plate": row["Metadata_Plate"],
                "well": row["Metadata_Well"],
                "site": row["Metadata_Site"],
                "location_center_x": row["Metadata_Nuclei_Location_Center_X"],
                "location_center_y": row["Metadata_Nuclei_Location_Center_Y"],
            }
    return sc_dict

In [3]:
# Function for generating and saving single-cell crops per channel as PNGs
def generate_sc_crops(
    sc_dict: dict,
    images_dir: pathlib.Path,
    output_img_dir: pathlib.Path,
    crop_size: int,
) -> None:
    """Using a dictionary with single-cell metadata info per image set, single-cell crops per channel are generated 
    and saved as PNGs in an image set folder.

    Args:
        sc_dict (dict): Dictionary containing info relevant for finding single-cell crops.
        images_dir (pathlib.Path): Directory where illumination corrected images are found.
        output_img_dir (pathlib.Path): Main directory to save each image set single-cell crops
        crop_size (int): Size of the box in pixels (example: setting crop_size as 250 will make a 250x250 pixel crop around the single-cell center coordinates)
    """
    for key, info in sc_dict.items():
        # Initialize a list to store file paths for every image set
        file_paths = []

        # Create file paths with well, site, and channel
        for i in range(5):  # Update the range to start from 0 and end at 4
            filename = f"{images_dir}/{info['plate']}_{info['well']}{info['site']}d{i}_illumcorrect.tiff"
            file_paths.append(filename)

            # Read the image
            channel_image = cv2.imread(filename, cv2.IMREAD_UNCHANGED)

            # Use the location_center_x and location_center_y to create a crop
            center_x = info.get("location_center_x")
            center_y = info.get("location_center_y")

            # Crop dimensions (including crop_size)
            half_crop = crop_size // 2

            # Ensure the center coordinates are valid
            if center_x is not None and center_y is not None:
                # Calculate crop boundaries
                top_left_x = max(int(center_x - half_crop), 0)
                top_left_y = max(int(center_y - half_crop), 0)
                bottom_right_x = min(int(center_x + half_crop), channel_image.shape[1])
                bottom_right_y = min(int(center_y + half_crop), channel_image.shape[0])

                # Perform cropping
                cropped_channel = channel_image[
                    top_left_y:bottom_right_y, top_left_x:bottom_right_x
                ]

                # Ensure the cropped image is of size 250x250
                cropped_channel = cv2.resize(cropped_channel, (crop_size, crop_size))

                # Make directory for the key to keep all channels for an image in one folder
                key_dir = pathlib.Path(f"{output_img_dir}/{key}")
                key_dir.mkdir(exist_ok=True, parents=True)

                # Save the cropped image with single_cell and channel information
                output_filename = str(pathlib.Path(f"{key_dir}/{key}_d{i}_cropped.png"))
                cv2.imwrite(output_filename, cropped_channel)

In [4]:
# Images directory for plate 4
images_dir = pathlib.Path(
    "../1.preprocessing_data/Corrected_Images/localhost231120090001/"
).resolve(strict=True)

# Output dir for cropped images
output_img_dir = pathlib.Path("./sc_crops")
output_img_dir.mkdir(exist_ok=True)

# Define the size of the cropping box (250x250 pixels)
crop_size = 250

# Create open list for one row data frames for each top feature per channel per cell type
list_of_dfs = []

# Create open list of names to assign each data frame in a list relating to the feature, channel, and cell type
list_of_names = []

In [5]:
# Load in normalized + feature selected data as data frame
plate4_df = pd.read_parquet(
    pathlib.Path(
        "../3.process_cfret_features/data/single_cell_profiles/localhost231120090001_sc_feature_selected.parquet"
    )
)

# Load in annotated dataframe to extract neighbors
annot_df = pd.read_parquet(
    pathlib.Path(
        "../3.process_cfret_features/data/single_cell_profiles/localhost231120090001_sc_annotated.parquet"
    ),
    columns=[
        "Metadata_Well",
        "Metadata_Site",
        "Metadata_Nuclei_Number_Object_Number",
        "Cells_Neighbors_NumberOfNeighbors_Adjacent",
    ],
)

plate4_df = plate4_df.merge(
    annot_df,
    on=["Metadata_Well", "Metadata_Site", "Metadata_Nuclei_Number_Object_Number"],
    how="inner",
)

plate4_df.rename(
    columns={
        "Cells_Neighbors_NumberOfNeighbors_Adjacent": "Metadata_Number_of_Cells_Neighbors_Adjacent"
    },
    inplace=True,
)

print(plate4_df.shape)
plate4_df.head()

(16887, 658)


,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_cell_type,Metadata_heart_failure_type,Metadata_treatment,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,...,Nuclei_Texture_InverseDifferenceMoment_ER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_00_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_02_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_03_256,Nuclei_Texture_SumVariance_ER_3_03_256,Nuclei_Texture_SumVariance_Hoechst_3_03_256,Nuclei_Texture_SumVariance_Mitochondria_3_03_256,Nuclei_Texture_SumVariance_PM_3_01_256,Metadata_Number_of_Cells_Neighbors_Adjacent
0,B,2,2,Healthy,None,None,199.395062,268.792593,193.914743,269.641581,...,0.692428,0.536752,0.878746,0.686496,0.963962,-0.324604,-0.291686,-0.238556,-0.351116,3.0
1,B,2,2,Healthy,None,None,303.540675,350.732143,314.842955,368.939934,...,0.605508,0.741645,0.972737,1.456869,1.470718,-0.301586,-0.305400,-0.246969,-0.376565,3.0
2,B,2,2,Healthy,None,None,700.388506,371.800000,708.112277,380.711178,...,1.314340,1.121741,1.188526,1.434736,1.417333,-0.343746,-0.358660,-0.252320,-0.383783,7.0
3,B,2,2,Healthy,None,None,855.869318,481.075758,877.161009,515.307396,...,0.084512,1.072561,0.900595,0.934998,0.540008,-0.271485,-0.305855,-0.240756,-0.278494,4.0
4,B,2,2,Healthy,None,None,586.624514,509.892023,590.453967,504.113587,...,0.608475,1.091661,1.060037,1.181420,0.982785,-0.330949,-0.263325,-0.253783,-0.356839,3.0


In [6]:
# Filter the DataFrame directly
filtered_plate4_df = plate4_df[
    (plate4_df['Metadata_Number_of_Cells_Neighbors_Adjacent'].isin([0])) &
    (plate4_df['Metadata_Nuclei_Location_Center_X'] > crop_size // 2) &
    (plate4_df['Metadata_Nuclei_Location_Center_X'] < (plate4_df['Metadata_Nuclei_Location_Center_X'].max() - crop_size // 2)) &
    (plate4_df['Metadata_Nuclei_Location_Center_Y'] > crop_size // 2) &
    (plate4_df['Metadata_Nuclei_Location_Center_Y'] < (plate4_df['Metadata_Nuclei_Location_Center_Y'].max() - crop_size // 2))
]

print(filtered_plate4_df.shape)
filtered_plate4_df.head()

(709, 658)


,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_cell_type,Metadata_heart_failure_type,Metadata_treatment,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,...,Nuclei_Texture_InverseDifferenceMoment_ER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_00_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_02_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_03_256,Nuclei_Texture_SumVariance_ER_3_03_256,Nuclei_Texture_SumVariance_Hoechst_3_03_256,Nuclei_Texture_SumVariance_Mitochondria_3_03_256,Nuclei_Texture_SumVariance_PM_3_01_256,Metadata_Number_of_Cells_Neighbors_Adjacent
12,B,2,2,Healthy,None,None,948.215054,804.736559,953.293162,787.218348,...,0.579334,0.382749,0.867683,0.165342,0.416402,-0.309225,-0.294345,-0.213416,-0.304800,0.0
15,B,2,2,Healthy,None,None,510.585419,138.967356,505.503133,151.434214,...,-1.576140,-1.891982,-3.522649,-1.303009,-1.694063,0.675887,0.268077,1.044633,0.255133,0.0
92,B,2,2,Healthy,None,None,220.169565,621.423913,228.558051,653.409680,...,0.584977,0.871408,0.965662,0.464366,0.394093,-0.318789,-0.167560,-0.223536,-0.316972,0.0
101,B,2,2,Healthy,None,None,733.387540,265.072862,717.630781,261.262095,...,0.430942,1.230734,1.427197,0.617736,0.420495,-0.335244,-0.242084,-0.243534,-0.288476,0.0
110,B,2,2,Healthy,None,None,757.728517,729.142310,763.741451,723.678065,...,0.424001,0.659869,0.398792,0.585751,0.737541,-0.268086,-0.138823,-0.227740,-0.314775,0.0


## Healthy features

In [7]:
# Get data frame with the top single-cell from the top healthy nuclei coefficient
top_healthy_nuclei = filtered_plate4_df[filtered_plate4_df["Metadata_cell_type"] == "Healthy"].nlargest(
    1, "Nuclei_Intensity_MeanIntensityEdge_Hoechst"
)[
    [
        "Nuclei_Intensity_MeanIntensityEdge_Hoechst",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Number_of_Cells_Neighbors_Adjacent",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_heart_number",
        "Metadata_cell_type",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(top_healthy_nuclei)
list_of_names.append("top_healthy_nuclei")

print(top_healthy_nuclei.shape)
top_healthy_nuclei

(1, 9)


,Nuclei_Intensity_MeanIntensityEdge_Hoechst,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Number_of_Cells_Neighbors_Adjacent,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_heart_number,Metadata_cell_type
16691,3.334627,G10,localhost231120090001,f09,0.0,185.040244,720.164634,7,Healthy


In [8]:
# Get data frame with the top single-cell from the top healthy nuclei coefficient
top_healthy_ER = filtered_plate4_df[filtered_plate4_df["Metadata_cell_type"] == "Healthy"].nlargest(
    1, "Cytoplasm_Texture_AngularSecondMoment_ER_3_01_256"
)[
    [
        "Cytoplasm_Texture_AngularSecondMoment_ER_3_01_256",
        "Metadata_cell_type",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_heart_number",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(top_healthy_ER)
list_of_names.append("top_healthy_ER")

print(top_healthy_ER.shape)
top_healthy_ER

(1, 8)


,Cytoplasm_Texture_AngularSecondMoment_ER_3_01_256,Metadata_cell_type,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_heart_number
11433,10.133427,Healthy,F02,localhost231120090001,f06,617.229803,444.958707,7


In [9]:
# Get data frame with the top single-cell from the top healthy nuclei coefficient
top_healthy_PM = filtered_plate4_df[filtered_plate4_df["Metadata_cell_type"] == "Healthy"].nlargest(
    1, "Nuclei_Intensity_IntegratedIntensity_PM"
)[
    [
        "Nuclei_Intensity_IntegratedIntensity_PM",
        "Metadata_cell_type",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_heart_number",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(top_healthy_PM)
list_of_names.append("top_healthy_PM")

print(top_healthy_PM.shape)
top_healthy_PM

(1, 8)


,Nuclei_Intensity_IntegratedIntensity_PM,Metadata_cell_type,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_heart_number
2142,4.495958,Healthy,B08,localhost231120090001,f09,129.173967,586.994368,2


In [10]:
# Get data frame with the top single-cell from the top healthy nuclei coefficient
top_healthy_mito = filtered_plate4_df[filtered_plate4_df["Metadata_cell_type"] == "Healthy"].nlargest(
    1, "Nuclei_RadialDistribution_MeanFrac_Mitochondria_3of4"
)[
    [
        "Nuclei_RadialDistribution_MeanFrac_Mitochondria_3of4",
        "Metadata_cell_type",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_heart_number",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(top_healthy_mito)
list_of_names.append("top_healthy_mito")

print(top_healthy_mito.shape)
top_healthy_mito

(1, 8)


,Nuclei_RadialDistribution_MeanFrac_Mitochondria_3of4,Metadata_cell_type,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_heart_number
9263,4.251189,Healthy,E04,localhost231120090001,f09,872.418622,175.130204,2


In [11]:
# Get data frame with the top single-cell from the top healthy nuclei coefficient
top_healthy_actin = filtered_plate4_df[filtered_plate4_df["Metadata_cell_type"] == "Healthy"].nlargest(
    1, "Nuclei_Intensity_MinIntensity_Actin"
)[
    [
        "Nuclei_Intensity_MinIntensity_Actin",
        "Metadata_cell_type",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_heart_number",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(top_healthy_actin)
list_of_names.append("top_healthy_actin")

print(top_healthy_actin.shape)
top_healthy_actin

(1, 8)


,Nuclei_Intensity_MinIntensity_Actin,Metadata_cell_type,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_heart_number
2081,0.810023,Healthy,B08,localhost231120090001,f01,419.17616,175.535865,2


## Failing features

In [12]:
# Get data frame with the top single-cell from the top healthy nuclei coefficient
top_failing_nuclei = filtered_plate4_df[filtered_plate4_df["Metadata_cell_type"] == "Failing"].nlargest(
    1, "Nuclei_Intensity_StdIntensityEdge_Hoechst"
)[
    [
        "Nuclei_Intensity_StdIntensityEdge_Hoechst",
        "Metadata_cell_type",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_heart_number",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(top_failing_nuclei)
list_of_names.append("top_failing_nuclei")

print(top_failing_nuclei.shape)
top_failing_nuclei

(1, 8)


,Nuclei_Intensity_StdIntensityEdge_Hoechst,Metadata_cell_type,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_heart_number
7795,0.57771,Failing,D08,localhost231120090001,f02,474.566225,734.282561,4


In [13]:
# Get data frame with the top single-cell from the top healthy nuclei coefficient
top_failing_ER = filtered_plate4_df[filtered_plate4_df["Metadata_cell_type"] == "Failing"].nlargest(
    1, "Cells_Intensity_IntegratedIntensityEdge_ER"
)[
    [
        "Cells_Intensity_IntegratedIntensityEdge_ER",
        "Metadata_cell_type",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_heart_number",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(top_failing_ER)
list_of_names.append("top_failing_ER")

print(top_failing_ER.shape)
top_failing_ER

(1, 8)


,Cells_Intensity_IntegratedIntensityEdge_ER,Metadata_cell_type,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_heart_number
2478,3.967654,Failing,B09,localhost231120090001,f15,664.381056,894.768223,19


In [14]:
# Get data frame with the top single-cell from the top healthy nuclei coefficient
top_failing_PM = filtered_plate4_df[filtered_plate4_df["Metadata_cell_type"] == "Failing"].nlargest(
    1, "Cells_Texture_InfoMeas1_PM_3_00_256"
)[
    [
        "Cells_Texture_InfoMeas1_PM_3_00_256",
        "Metadata_cell_type",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_heart_number",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(top_failing_PM)
list_of_names.append("top_failing_PM")

print(top_failing_PM.shape)
top_failing_PM

(1, 8)


,Cells_Texture_InfoMeas1_PM_3_00_256,Metadata_cell_type,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_heart_number
10240,4.8923,Failing,E07,localhost231120090001,f06,280.077328,672.516044,19


In [15]:
# Get data frame with the top single-cell from the top healthy nuclei coefficient
top_failing_mito = filtered_plate4_df[filtered_plate4_df["Metadata_cell_type"] == "Failing"].nlargest(
    1, "Nuclei_Intensity_LowerQuartileIntensity_Mitochondria"
)[
    [
        "Nuclei_Intensity_LowerQuartileIntensity_Mitochondria",
        "Metadata_cell_type",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_heart_number",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(top_failing_mito)
list_of_names.append("top_failing_mito")

print(top_failing_mito.shape)
top_failing_mito

(1, 8)


,Nuclei_Intensity_LowerQuartileIntensity_Mitochondria,Metadata_cell_type,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_heart_number
9558,2.929069,Failing,E05,localhost231120090001,f09,261.589552,383.65796,19


In [16]:
# Get data frame with the top single-cell from the top healthy nuclei coefficient
top_failing_actin = filtered_plate4_df[filtered_plate4_df["Metadata_cell_type"] == "Failing"].nlargest(
    1, "Cells_Intensity_IntegratedIntensityEdge_Actin"
)[
    [
        "Cells_Intensity_IntegratedIntensityEdge_Actin",
        "Metadata_cell_type",
        "Metadata_Well",
        "Metadata_Plate",
        "Metadata_Site",
        "Metadata_Nuclei_Location_Center_X",
        "Metadata_Nuclei_Location_Center_Y",
        "Metadata_heart_number",
    ]
]

# Append the DataFrame and its name to the lists
list_of_dfs.append(top_failing_actin)
list_of_names.append("top_failing_actin")

print(top_failing_actin.shape)
top_failing_actin

(1, 8)


,Cells_Intensity_IntegratedIntensityEdge_Actin,Metadata_cell_type,Metadata_Well,Metadata_Plate,Metadata_Site,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_heart_number
7964,11.574613,Failing,D09,localhost231120090001,f03,655.204698,682.96896,29


In [17]:
sc_dict = create_sc_dict(dfs=list_of_dfs, names=list_of_names)

# Check the created dictionary for the first two items
pprint(list(sc_dict.items())[:2], indent=4)

[   (   'top_healthy_nuclei',
        {   'location_center_x': 185.04024390243902,
            'location_center_y': 720.1646341463414,
            'plate': 'localhost231120090001',
            'site': 'f09',
            'well': 'G10'}),
    (   'top_healthy_ER',
        {   'location_center_x': 617.229802513465,
            'location_center_y': 444.95870736086175,
            'plate': 'localhost231120090001',
            'site': 'f06',
            'well': 'F02'})]


In [18]:
generate_sc_crops(sc_dict=sc_dict, images_dir=images_dir, output_img_dir=output_img_dir, crop_size=crop_size)